# Recitation 3

Topics: ASTs, higher order functions, pattern functions

## Lists and Pattern Matching

Our inductive structure of choice for this problem is a linked list. The grammar below defines the structure.
$$
\begin{array}{rcl}
    \textbf{NumList} & \rightarrow & MyNil \\
                     &           | & MyCons(\textbf{Int}, \textbf{NumList}) \\
\end{array}
$$

### Exercise: Write the Scala Types

Define the Scala classes based on this grammar.

In [1]:
sealed trait NumList
case object MyNil extends NumList
case class MyCons(i: Int, l: NumList) extends NumList

defined trait NumList
defined object MyNil
defined class MyCons

### Exercise: Writing lists
Write out the following lists with our `NumList` class.

1. `[]` (The empty list)
2. `[1, 2, 3]`

In [2]:
val ans_1 = MyNil
val ans_2 = MyCons(1, MyCons(2, MyCons(3, MyNil)))

ans_1: MyNil = MyNil
ans_2: MyCons = MyCons(1, MyCons(2, MyCons(3, MyNil)))

### Exercise: Length of list
Implement a `myLength` function for our list type using [pattern matching](https://docs.scala-lang.org/tour/pattern-matching.html).

In [3]:
def myLength(list : NumList) : Int = {
    list match {
        case MyNil => {
            0
        }
        case MyCons(_, yaas) => {
            myLength(yaas) + 1
        }
    }
}

defined function myLength

In [4]:
assert(myLength(MyNil) == 0)
assert(myLength(MyCons(1, MyCons(2, MyNil))) == 2)

### Exercise: Translate to built in lists
Rewrite `length` to use [Scala's list class](https://www.scala-lang.org/api/current/scala/collection/immutable/List.html), which is very similar to the one defined above. The table below shows the equivalences:

|`NumList`      | `List[Int]`|
|---------------|------------|
|`MyNil`        | `Nil`      |
|`MyCons(a, b)` | `a :: b`   |

In [5]:
def length(list : List[Int]) : Int = {
    list match {
        case Nil => 0
        case _::b => length(b) + 1
    }
}

defined function length

In [6]:
assert(length(List()) == 0)
assert(length(List(1, 2, 3)) == 3)

### Exercise: Count the Number of Even Elements
Write a function that counts the number of even elements in the list.

E.g. countEven(List(1,2,3,4,5)) = 2

In [7]:
def countEven(list:List[Int]):Int = {
    list match {
        case Nil => 0
        case a::b => {
            if (a % 2 == 0) {
                countEven(b) + 1
            } else {
                countEven(b)
            }
        }
    }
}

defined function countEven

In [8]:
assert(countEven(List(1,2,3,4,5)) == 2)
assert(countEven(List()) == 0)

### Exercise: Drop Until Size is Reached
Write a function that takes a list and a number.  The function removes elements from the front of the list until the list is a given length.

E.g. dropN(List(1,2,3,4), 2) = List(3,4)

In [12]:
def dropN(list:List[Int], n: Int):List[Int] = {
    list match {
        case Nil => Nil
        case a::b => {
            if (n <= 0) {
                a::b
            } else {
                dropN(b, n-1)
            }
        }
    }
}

defined function dropN

In [13]:
assert(dropN(List(1,2,3,4), 2) == List(3,4))
assert(dropN(List(), 4) == List())

### Exercise: Higher Order functions
Define a function called `map`. `map` should takes a list of `Int`s, and a function from `Int` to `Int`, then calls the function on each element and makes a new list from the results.

For example, the following call:
```
map(List(1, 2, 3, 4, 5), (x) => x + 1)
```
should return in the following list
```
List(2, 3, 4, 5, 6)
```

In [16]:
def map(ls: List[Int], f: (Int => Int)): List[Int] = {
    ls match {
        case Nil => Nil
        case a::b => {
            f(a)::map(b, f)
        }
    }
}

defined function map

In [17]:
assert(map(List(), _ + 1) == List())
assert(map(List(1), _ + 1) == List(2))
assert(map(List(1, 9, 2), _ + 1) == List(2, 10, 3))

## ASTs
We'll be reusing the AST for sets from the previous recitation (removing the complement operator). Use Scala lists as your underlying datatype to represent sets (even though it won't be very efficient):

$$
\begin{array}{rcl}
    \textbf{Set} & \rightarrow & Empty \\
                 &           | & Cons(\textbf{Int}, \textbf{Set}) \\
                 &           | & Intersection(\textbf{Set}, \textbf{Set}) \\
                 &           | & Union(\textbf{Set}, \textbf{Set}) \\
                 &           | & Subtraction(\textbf{Set}, \textbf{Set}) \\
\end{array}
$$

One last little note: $\textbf{Int}$ and $\textbf{Set}$ are switched from last week to match with Scala's `::` and the lecture.

In [18]:
sealed trait Set
case object Empty extends Set
case class Cons(n: Int, s: Set) extends Set
// Removed
// case class Complement(s: Set) extends Set
case class Intersection(s1: Set, s2: Set) extends Set
case class Union(s1: Set, s2: Set) extends Set
case class Subtraction(s1: Set, s2: Set) extends Set

defined trait Set
defined object Empty
defined class Cons
defined class Intersection
defined class Union
defined class Subtraction

### Exercise: Interpreter
Write an interpreter for the language of sets you've created. Don't worry about duplicates in the list (unless they need to be removed).

```
List.++:       (List[Int], List[Int])      => List[Int]
List.filter:   (List[Int], Int => Boolean) => List[Int]
List.contains: (List[Int], Int)            => Boolean
```

In [26]:
def eval(se: Set): List[Int] = {
    se match {
        case Empty => Nil
        case Cons(i, s) => i::eval(s)
        case Intersection(s1, s2) => {
            val tmp = eval(s1)
            eval(s2).filter(x => tmp.contains(x))
        }
        case Union(s1, s2) => {
            // does not union correctly
            eval(s1) ++ eval(s2)
        }
        case Subtraction(s1, s2) => {
            val tmp1 = eval(s1)
            val tmp2 = eval(s2)
            tmp1.filter(x => !tmp2.contains(x))
        }
    }
}

defined function eval

In [28]:
val set_1_2 = Cons(1, Cons(2, Empty))
val set_1_2_3 = Cons(1, Cons(2, Cons(3, Empty)))
val set_3 = Cons(3, Empty)

assert(eval(Empty) == List())
assert(eval(set_1_2) == List(1, 2))
assert(eval(Union(set_1_2, set_3)) == List(1, 2, 3))
assert(eval(Intersection(set_1_2, set_1_2_3)) == List(1, 2))
assert(eval(Subtraction(set_1_2, set_1_2_3)) == List())

set_1_2: Cons = Cons(1, Cons(2, Empty))
set_1_2_3: Cons = Cons(1, Cons(2, Cons(3, Empty)))
set_3: Cons = Cons(3, Empty)